In [1]:
import random
import mne, os, glob
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import random
from mne.datasets import somato
from mne.time_frequency import tfr_morlet
%matplotlib qt 

In [ ]:
""" 
create data path
"""
# Create data paths 
sub = 4
cleaned_data_dir = '/Users/mvmigem/Documents/data/project_1/preprocessed/mastoid_ref/'
clean_epoch_path = os.path.join(cleaned_data_dir,f'main_eventset_mastoidref_{sub:02}-epo.fif')
eps = mne.read_epochs(clean_epoch_path)

In [2]:
ep_list = []
for sub in range(1,15):
    cleaned_data_dir = '/Users/mvmigem/Documents/data/project_1/preprocessed/mastoid_ref/'
    clean_epoch_path = os.path.join(cleaned_data_dir,f'main_eventset_mastoidref_{sub:02}-epo.fif')
    epochs = mne.read_epochs(clean_epoch_path)
    epochs.info['bads']=[]
    ep_list.append(epochs)
eps = mne.concatenate_epochs(ep_list)

Reading c:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\main_eventset_mastoidref_01-epo.fif ...
    Found the data of interest:
        t =     -97.66 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 33 columns
4189 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\main_eventset_mastoidref_02-epo.fif ...
    Found the data of interest:
        t =     -97.66 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 33 columns
4196 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mvmigem\Documents\data\project_1\preprocessed\mastoid_ref\main_eventset_mastoidref_03-epo.fif ...
    Found the data of interest:
        t =     -97.66 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 32 columns
4188 matching events found
N

C:\Users\mvmigem\AppData\Local\Temp\ipykernel_4192\1887482926.py:8: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  eps = mne.concatenate_epochs(ep_list)


Adding metadata with 35 columns
56855 matching events found
Applying baseline correction (mode: mean)


In [3]:



# Divide the epoch file into sections based on metadata that can't be distinguished by event names
# Drop catch trials
epochs_nocatch = eps['catch_trial == 0']
# Divide attention conditions
epochs_attended = epochs_nocatch['attention == "attended"']
epochs_unattended = epochs_nocatch['attention == "unattended"']
# Divide by staring position
epochs_start3 = epochs_nocatch['start_position == 2']
epochs_start1 = epochs_nocatch['start_position == 0']
epochs_start2 = epochs_nocatch['start_position == 1']
epochs_start4 = epochs_nocatch['start_position == 3']
# Divide unattended trials by start pos
epochs_unattended_start3 = epochs_unattended['start_position == 2']
epochs_unattended_start1 = epochs_unattended['start_position == 0']
epochs_unattended_start2 = epochs_unattended['start_position == 1']
epochs_unattended_start4 = epochs_unattended['start_position == 3']
# Divide attented trials by start pos
epochs_attended_start3 = epochs_attended['start_position == 2']
epochs_attended_start1 = epochs_attended['start_position == 0']
epochs_attended_start2 = epochs_attended['start_position == 1']
epochs_attended_start4 = epochs_attended['start_position == 3']
# Extra devision for the odd boys
epochs_attended_reg = epochs_attended['expected =="regular"']
epochs_attended_odd = epochs_attended['expected == "odd"']
epochs_unattended_reg = epochs_unattended['expected =="regular"']
epochs_unattended_odd = epochs_unattended['expected == "odd"']

In [ ]:
big_meta = eps.metadata

In [ ]:
"""
Main pos topo
"""
times = np.linspace(0.03, 0.15, 7)
vlim=(-6, 6)

epochs_nocatch['pos1'].average().plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)
epochs_nocatch['pos2'].average().plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)
epochs_nocatch['pos3'].average().plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)
epochs_nocatch['pos4'].average().plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)

### Evoked plotting

In [ ]:
"""
Main pos individual channels
"""
evoked_pos1 = epochs_nocatch['pos1'].average()
evoked_pos2 = epochs_nocatch['pos2'].average()
evoked_pos3 = epochs_nocatch['pos3'].average()
evoked_pos4 = epochs_nocatch['pos4'].average()

evokeds_list = [evoked_pos1,evoked_pos2,evoked_pos3,evoked_pos4]
conds = ('pos1','pos2','pos3','pos4')
# conds = ('seq1','seq2','seq3','seq4')

norm = dict(zip(conds, evokeds_list))

In [ ]:
""" 
2: attention x predicion 
"""
ep_reg_att = epochs_attended_start4['precedes_odd == 1']['pos3/seq2']
ep_odd_att = epochs_attended_start2['pos3/seq2']
ep_reg_unatt = epochs_unattended_start4['precedes_odd == 1']['pos3/seq2']
ep_odd_unatt = epochs_unattended_start2['pos3/seq2']

In [4]:
""" 
prediction independent plots
"""

ep_att_1 = epochs_attended_start1['seq']
ep_att_2 = epochs_attended_start2['seq1']
ep_att_3 = epochs_attended_start3['seq1']
ep_att_4 = epochs_attended_start4['seq1']

ep_unatt_1 = epochs_unattended_start1['seq']
ep_unatt_2 = epochs_unattended_start2['seq1']
ep_unatt_3 = epochs_unattended_start3['seq1']
ep_unatt_4 = epochs_unattended_start4['seq1']

In [ ]:
"""
prediction v attention
"""
evoked_pos1 = ep_reg_att.average()
evoked_pos2 = ep_odd_att.average()
evoked_pos3 = ep_reg_unatt.average()
evoked_pos4 = ep_odd_unatt.average()

evokeds_list = [evoked_pos1,evoked_pos2,evoked_pos3,evoked_pos4]
conds = ('att reg','att odd','unatt reg','unatt odd')
norm = dict(zip(conds, evokeds_list))

In [5]:
"""
attention effect 1 in sequence
"""
evoked_pos1 = ep_att_1.average()
evoked_pos2 = ep_att_2.average()
evoked_pos3 = ep_att_3.average()
evoked_pos4 = ep_att_4.average()
evoked_pos5 = ep_unatt_1.average()
evoked_pos6 = ep_unatt_2.average()
evoked_pos7 = ep_unatt_3.average()
evoked_pos8 = ep_unatt_4.average()

evokeds_list = [evoked_pos1,evoked_pos2,evoked_pos3,evoked_pos4, evoked_pos5, evoked_pos6,evoked_pos7,evoked_pos8]
conds = ('att 1','att 2','att 3','att 4','unatt 1','unatt 2','unatt 3','unatt 4')
norm = dict(zip(conds, evokeds_list))

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [ ]:
""" 
Pos 1 and 3
"""

evokeds_list = [evoked_pos1,evoked_pos3, evoked_pos5,evoked_pos7]
conds = ('att 1','att 3','unatt 1','unatt 3')
norm = dict(zip(conds, evokeds_list))

In [ ]:
""" 
Pos 2 and 4
"""

evokeds_list = [evoked_pos2,evoked_pos4, evoked_pos6,evoked_pos8]
conds = ('att 2','att 4','unatt 2','unatt 4')
norm = dict(zip(conds, evokeds_list))

In [ ]:
"""
attention effect 1 in sequence
"""

evoked_pos1 = ep_att_4['trial < 211'].average()
evoked_pos2 = ep_att_4['210 < trial < 421'].average()
evoked_pos3 = ep_att_4['420 < trial < 631'].average()
evoked_pos4 = ep_att_4['trial > 630'].average()

evokeds_list = [evoked_pos1,evoked_pos2,evoked_pos3,evoked_pos4]
conds = ('att 1','att 2','att 3','att 4')
norm = dict(zip(conds, evokeds_list))

In [ ]:
rrr= ep_att_4['210 < trial < 421']

In [ ]:
"""
sequence evaluation
"""
evoked_pos1 = ep_reg_p4s2.average()
evoked_pos2 = ep_reg_p4s4.average()
evoked_pos3 = ep_reg_p2s2.average()
evoked_pos4 = ep_reg_p2s4.average()

evokeds_list = [evoked_pos1,evoked_pos2,evoked_pos3,evoked_pos4]
conds = ('p4s2','p4s4','p2s2','p2s4')
norm = dict(zip(conds, evokeds_list))

In [6]:
epoch_set1 = norm
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'POz', vlines=[0.05,0.1],ylim=dict(eeg=[-10, 10]))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'Pz', vlines=[0.05,0.1],ylim=dict(eeg=[-10, 10]))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'Oz', vlines=[0.05,0.1],ylim=dict(eeg=[-10, 10]))


[<Figure size 800x600 with 2 Axes>]

In [ ]:
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'O1', vlines=[0.05,0.1],ylim=dict(eeg=[-6, 6]))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'O2', vlines=[0.05,0.1],ylim=dict(eeg=[-6, 6]))

In [ ]:
# mne.viz.plot_compare_evokeds(epoch_set1, picks= 'P1', vlines=[0.05,0.1],ylim=dict(eeg=[-6, 6]))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'P2', vlines=[0.05,0.1],ylim=dict(eeg=[-6, 6]))

In [ ]:
# mne.viz.plot_compare_evokeds(epoch_set1, picks= 'PO2', vlines=[0.05,0.1],ylim=dict(eeg=[-10, 10]))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'PO3', vlines=[0.05,0.1],ylim=dict(eeg=[-10, 10]))

In [ ]:
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'Fz', vlines=[0.05,0.1],ylim=dict(eeg=[-6, 6]))
mne.viz.plot_compare_evokeds(epoch_set1, picks= 'FCz', vlines=[0.05,0.1],ylim=dict(eeg=[-6, 6]))

In [ ]:
"""

"""
times = np.linspace(0.03, 0.15, 7)
vlim=(-6, 6)

evoked_pos1.plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)
evoked_pos2.plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)
evoked_pos3.plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)
evoked_pos4.plot_topomap(ch_type="eeg", times=times, colorbar=True, vlim=vlim)

### Time course of c1 peaks and means

In [7]:
""" 
Time course first sequence position
"""
# attended
ep_df_att_1 = ep_att_1.metadata
ep_df_att_1['poz_c1_mean'] = ep_att_1.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_att_2 = ep_att_2.metadata
ep_df_att_2['poz_c1_mean'] = ep_att_2.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_att_3 = ep_att_3.metadata
ep_df_att_3['poz_c1_mean'] = ep_att_3.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_att_4 = ep_att_4.metadata
ep_df_att_4['poz_c1_mean'] = ep_att_4.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
# unattend
ep_df_unatt_1= ep_unatt_1.metadata
ep_df_unatt_1['poz_c1_mean'] = ep_unatt_1.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_unatt_2 = ep_unatt_2.metadata
ep_df_unatt_2['poz_c1_mean'] = ep_unatt_2.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_unatt_3 = ep_unatt_3.metadata
ep_df_unatt_3['poz_c1_mean'] = ep_unatt_3.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)
ep_df_unatt_4 = ep_unatt_4.metadata
ep_df_unatt_4['poz_c1_mean'] = ep_unatt_4.crop(tmin=.07, tmax=.1).get_data(picks='POz').mean(axis=2)


In [ ]:
""" 
Time course
"""
position = ['pos1','pos2','pos3','pos4']
condition = 'attention == "attended" & expected == "regular" & start_position == 2'
pos1_win_data = c1_window_epochs[condition][position[0]].get_data(picks='POz')
pos2_win_data = c1_window_epochs[condition][position[1]].get_data(picks='POz')
pos3_win_data = c1_window_epochs[condition][position[2]].get_data(picks='POz')
pos4_win_data = c1_window_epochs[condition][position[3]].get_data(picks='POz')

In [ ]:
mean_c1_1 = pos1_win_data.mean(axis=2).squeeze()
mean_c1_2 = pos2_win_data.mean(axis=2).squeeze()
mean_c1_3 = pos3_win_data.mean(axis=2).squeeze()
mean_c1_4 = pos4_win_data.mean(axis=2).squeeze()

max_c1_1 = pos1_win_data.max(axis=2).squeeze()
max_c1_2 = pos2_win_data.max(axis=2).squeeze()
max_c1_3 = pos3_win_data.max(axis=2).squeeze()
max_c1_4 = pos4_win_data.max(axis=2).squeeze()

mean_c1_list = [mean_c1_1,mean_c1_2,mean_c1_3,mean_c1_4]
max_c1_list = [max_c1_1,max_c1_2,max_c1_3,max_c1_4]

In [10]:
"""
Plotting mean c1 voltage time wise
"""
plt.rcParams["figure.figsize"] = [18,10]
fig, axs = plt.subplots(4,2, sharey=True)

sns.regplot(data = ep_df_att_1, x = 'trial', y = 'poz_c1_mean', ax = axs[0,0],scatter_kws={'s': 10, 'color': 'red'})
sns.regplot(data = ep_df_unatt_1, x = 'trial', y = 'poz_c1_mean', ax = axs[1,0],scatter_kws={'s': 10, 'color': 'green'})
sns.regplot(data = ep_df_att_2, x = 'trial', y = 'poz_c1_mean', ax = axs[0,1],scatter_kws={'s': 10, 'color': 'red'})
sns.regplot(data = ep_df_unatt_2, x = 'trial', y = 'poz_c1_mean', ax = axs[1,1],scatter_kws={'s': 10, 'color': 'green'})
sns.regplot(data = ep_df_att_3, x = 'trial', y = 'poz_c1_mean', ax = axs[2,1],scatter_kws={'s': 10, 'color': 'red'})
sns.regplot(data = ep_df_unatt_3, x = 'trial', y = 'poz_c1_mean', ax = axs[3,1],scatter_kws={'s': 10, 'color': 'green'})
sns.regplot(data = ep_df_att_4, x = 'trial', y = 'poz_c1_mean', ax = axs[2,0],scatter_kws={'s': 10, 'color': 'red'})
sns.regplot(data = ep_df_unatt_4, x = 'trial', y = 'poz_c1_mean', ax = axs[3,0],scatter_kws={'s': 10, 'color': 'green'})
plt.tight_layout()
plt.show()
# sns.regplot(x = np.arange(len(mean_c1_list[1])), y = mean_c1_list[1], ax=axs[0,1])
# sns.regplot(x = np.arange(len(mean_c1_list[2])), y = mean_c1_list[2], ax=axs[1,1])
# sns.regplot(x = np.arange(len(mean_c1_list[3])), y = mean_c1_list[3], ax=axs[1,0])



In [ ]:
"""
Plotting max c1 voltage time wise
"""
plt.rcParams["figure.figsize"] = [18,10]
fig, axs = plt.subplots(2,2, sharey=True)

sns.regplot(x = np.arange(len(max_c1_list[0])), y = max_c1_list[0], ax=axs[0,0])
sns.regplot(x = np.arange(len(max_c1_list[1])), y = max_c1_list[1], ax=axs[0,1])
sns.regplot(x = np.arange(len(max_c1_list[2])), y = max_c1_list[2], ax=axs[1,1])
sns.regplot(x = np.arange(len(max_c1_list[3])), y = max_c1_list[3], ax=axs[1,0])

plt.suptitle('max values '+ condition)
plt.tight_layout()
plt.show()

### Time frequency exploration

In [ ]:
spec_first_att = epochs_attended['pos4/seq2'].average().compute_psd()
spec_last_att = epochs_attended['seq3'].average().compute_psd()
spec_first_unatt = epochs_unattended['seq1'].average().compute_psd()
spec_last_unatt = epochs_unattended['seq3'].average().compute_psd()

In [ ]:
spec_first_att.plot_topomap(ch_type="eeg")
spec_first_unatt.plot_topomap(ch_type="eeg")

In [ ]:
freqs = np.logspace(*np.log10([8, 13]), num=8)
n_cycles = freqs / 3.0  # different number of cycle per frequency
power, itc = tfr_morlet(
    epochs_attended['pos4/seq2'],
    freqs=freqs,
    n_cycles=n_cycles,
    use_fft=True,
    return_itc=True,
    n_jobs=None,
)

In [ ]:

power.plot([27], baseline=(-0.09, 0), mode="logratio", title=power.ch_names[27])

fig, axes = plt.subplots(1, 2, figsize=(7, 4), layout="constrained")
topomap_kw = dict(
    ch_type="eeg", tmin=0.09, tmax=0.5, baseline=(-0.09, 0), mode="logratio", show=False
)
plot_dict = dict(Alpha=dict(fmin=8, fmax=13))
for ax, (title, fmin_fmax) in zip(axes, plot_dict.items()):
    power.plot_topomap(**fmin_fmax, axes=ax, **topomap_kw)
    ax.set_title(title)